In [1]:
'''
#########################################################################################################################
INFORMACIÓN DEL FICHERO
#########################################################################################################################

Autor: Samuel Lozano Juárez
Fecha: 24/05/2023
Institución: UBU | Grado en Ingeniería de la Salud

Este archivo forma parte del Trabajo de Fin de Grado "Detección del grado de retinopatía mediante redes convolucionales".
El alumno a cargo de este proyecto es el declarado como autor en las líneas anteriores.
Los tutores del proyecto fueron el Dr. Darío Fernández Zoppino y el Dr. Daniel Urda Muñoz.

En el código que se encuentra a continuación voy a crear una primera arquitectura de red neuronal convolucional, así como las demás 
estructuras necesarias para poder entrenar dicha red (como por ejemplo las funciones que permiten cargar las imágenes o generar los 
batches o lotes para el entrenamiento).

El entrenamiento de la red se llevará a cabo empleando un conjunto de validación para poder aplicar la estrategia de Early Stopping y evitar el sobreentrenamiento.

Para entrenar los modelos se usarán imágenes de OCT + Samsung o iPhone y se testearán con el conjunto de imágenes no empleado en el entrenamiento (iPhone o Samsung respectivamente).

La arquitectura del modelo será básica, siguiendo la estructura que se encuentra disponible en la propia página del framework Pytorch:
https://pytorch.org/tutorials/beginner/introyt/introyt1_tutorial.html#pytorch-models

Finalmente se obtendrán las métricas de Accuracy, Balanced Accuracy, F-Score, AUC de la curva ROC y Quadratic Weighted Kappa, tanto para
imágenes de iPhone como imágenes de Samsung. De esta manera podremos evaluar el rendimiento de la red en comparación con un clínico.
'''

#primero importamos todos los paquetes necesarios
import torch #contiene todas las funciones de PyTorch
import torch.nn as nn #contiene la clase padre de todos los modelos (nn.Module)
import torch.nn.functional as F #esencial para la función de activación 
import torchvision #fundamental para la importación de imágenes
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset
from matplotlib import pyplot as plt #para poder representar las gráficas
import numpy as np #para las métricas de la red

#importamos también las funciones definidas para el entrenamiento y puesta a prueba de los modelos
from modules.CNN_utilities import entrena_val, representa_test, obtiene_metricas, tester, guarda_graficas

#importamos el paquete que permite calcular el tiempo de entrenamiento
import time

C:\Users\samue\anaconda3\envs\venv_py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#establecemos el tamaño del batch, la escala de las imágenes y el número de épocas de entrenamiento
batch = 4
escala = 640
epocas = 150 #ya que tenemos activado Early Stopping

#a continuación definimos la operación que permitirá transformar las imágenes del repositorio en Tensores que puedan ser empleados por PyTorch
transform = transforms.Compose(
    [transforms.ToTensor(), #transforma la imagen de formato PIL a formato tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #normaliza el tensor para que la media de sus valores sea 0 y su desviación estándar 0.5
     transforms.Resize((escala, escala))]) #redimensionamos las imágenes

In [ ]:
#COMENZAMOS CON LOS ELEMENTOS NECESARIOS PARA EL ENTRENAMIENTO CON SAMSUNG Y TEST CON IPHONE
#primero definimos una lista con las rutas de los directorios que queremos combinar (OCT + Samsung)
root_dirs_OCT_S = ['Datos/Classified Data/Images/Samsung/No_inpaint', 'Datos/Classified Data/Images/OCT']
#inicializamos una lista 'datasets_OCT_S' vacía que almacenará las imágenes y etiquetas
datasets_OCT_S = []
#recorremos los directorios a concatenar
for root_dir in root_dirs_OCT_S:
    #cargamos las imágenes (y etiquetas correspondientes) de dichos directorios, con la transformación aplicada
    dataset = ImageFolder(root_dir, transform = transform)
    #añadimos esas imágenes y etiquetas a la lista previamente creada
    datasets_OCT_S.append(dataset)

#concatenamos esas imágenes y mostramos el tamaño total del dataset de entrenamiento
OCT_S = ConcatDataset(datasets_OCT_S)
print(f'Tamaño del conjunto de datos de train: {len(OCT_S)}')

#cargamos el dataset de test y mostramos su tamaño
iPhone = ImageFolder(root = 'Datos/Classified Data/Images/iPhone/No_inpaint', transform = transform)
print(f'Tamaño del conjunto de datos de test de iPhone: {len(iPhone)}')

In [ ]:
#en esta ocasión, debido a que vamos a implementar EarlyStopping es necesario dividir el conjunto de entrenamiento en train y validation
#Dividimos el conjunto de datos en entrenamiento y validación (80% y 20% respectivamente)
train_size = int(0.8 * len(OCT_S))
val_size = len(OCT_S) - train_size
train_dataset_OCT_S, val_dataset_OCT_S = torch.utils.data.random_split(OCT_S, [train_size, val_size])

In [ ]:
#finalmente creamos los objetos DataLoader correspondientes (de entrenamiento, validación y test con iPhone)
#primero el DataLoader de entrenamiento
train_loader_OCT_S = DataLoader(
    train_dataset_OCT_S, #indicamos el conjunto de imágenes combinadas de entrenamiento
    batch_size=batch,
    shuffle=True, #mezclamos las imágenes para combinar de todos los grados y fuentes en cada batch
    num_workers = 2 #así genera subprocesos y acelera la alimentación del modelo con imágenes
)

#posteriormente definimos el DataLoader de validación
val_loader_OCT_S = DataLoader(
    dataset = val_dataset_OCT_S, #indicamos el conjunto de imágenes combinadas de validación
    batch_size = batch,
    shuffle = True, #mezclamos las imágenes para combinar de todos los grados y fuentes en cada batch
    num_workers = 2 #así genera subprocesos y acelera la alimentación del modelo con imágenes
)

#y finalmente el DataLoader de test con las imágenes de iPhone
test_i_loader = DataLoader(
    dataset = iPhone,
    batch_size = batch,
    shuffle = True, #mezclamos las imágenes para combinar de todos los grados y fuentes en cada batch
    num_workers = 2 #así genera subprocesos y acelera la alimentación del modelo con imágenes
)

In [ ]:
#AHORA REALIZAMOS EL MISMO PROCESO PERO PARA ENTRENAR EL MODELO CON IPHONE Y HACER EL TEST CON SAMSUNG